# Escalas, Eixos e Legendas
---

Codificação visual &ndash; mapear os dados em variáveis visuais como posição, tamanho, formato ou cor &ndash; é o coração pulsante da visualização de dados. O cavalo-potência que de fato protagoniza esse mapeamento é a *escala*: uma função que toma um valor de dados como *input* (o *domínio* da escala) e retorna um valor visual, como a posição de um pixel ou cor RGB, como *output* (a *extensão* ou *escopo* da escala). É claro que uma visualização é inútil se ninguém a entende. Adicionalmemte às marcas gráficas, um gráfico carece de elementos de referência, ou *guias*, que permite que seus leitores o decodifiquem. Guias como *eixos* (que expressam escalas de maneira espacial) e *legendas* (que complementam a informação com cor, tamanho ou formato) são os heróis esquecidos da visualização de dados!

Nesse capítulo, vamos explorar as opções oferecidas pelo Altair para possibilitar designs customizados de mapeamento de escala, eixo e legendas, através do exemplo da eficácia de antibióticos.

_Este capítulo faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [ ]:
import pandas as pd
import altair as alt

## Dados de antibióticos
---

Após a segunda guerra mundial, antibióticos foram considerados "drogas milagrosas", à medida que remediavam com facilidade males antes incuráveis. Para compreender qual droga seria mais eficaz contra cada infecção bacteriana, acumulamos dada acerca da performance dos três antibióticos mais populares em relação a dezesseis bactérias.

Usaremos um dataset de antibióticos do [vega-datasets collection](https://github.com/vega/vega-datasets). Nos exemplos abaixo, há referência direta do URL para Altair:

In [ ]:
antibiotics = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/burtin.json'

Podemos primeiro ler os dados com Pandas para observar o dataset em toda a sua extensão e tomar ciência dos campos disponíveis:


In [ ]:
pd.read_json(antibiotics)

,Bacteria,Penicillin,Streptomycin,Neomycin,Gram_Staining,Genus
0,Aerobacter aerogenes,870.000,1.00,1.600,negative,other
1,Bacillus anthracis,0.001,0.01,0.007,positive,other
2,Brucella abortus,1.000,2.00,0.020,negative,other
3,Diplococcus pneumoniae,0.005,11.00,10.000,positive,other
4,Escherichia coli,100.000,0.40,0.100,negative,other
5,Klebsiella pneumoniae,850.000,1.20,1.000,negative,other
6,Mycobacterium tuberculosis,800.000,5.00,2.000,negative,other
7,Proteus vulgaris,3.000,0.10,0.100,negative,other
8,Pseudomonas aeruginosa,850.000,2.00,0.400,negative,other
9,Salmonella (Eberthella) typhosa,1.000,0.40,0.008,negative,Salmonella


Os valores numéricos presentes na tabela indicam a [mínima concentração inibitória (MIC)](https://en.wikipedia.org/wiki/Minimum_inhibitory_concentration), que mede a eficácia do antibiótico em relação à sua concentração (em microgramas por mililitro) requerida para prevenir o crescimento da colônia in vitro. A reação da bactéria a um processo chamado [Técnica de Gram](https://en.wikipedia.org/wiki/Gram_stain) é descrito pelo campo de nome `Gram_Staining`. As bactérias que adiquirem coloração num tom escuro de azul ou roxo são Gram-positive. O restante é Gram-negative.

Enquanto examinamos as diferentes visualizações desse dataset, se pergunte: o que podemos aprender sobre a eficácia relativa dos antibióticos? E sobre as espécies bacterianas com base na sua "resposta imunológica"?

## Configurando escalas e eixos
---

### Plotando resistência anibiótica: ajustagem do tipo de escala
---

Comecemos olhando um diagrama de pontos simples para o MIC da Neomicina.

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q')
)

alt.Chart(...)

_Podemos ver que os valores MIC se distribuem da seguinte forma: a maioria se acumula à esquerda, com uns poucos outliers à direita._

Por padrão, o Altair usa um mapeamento `linear` entre valores do domínio (MIC) e valores do intervalo (pixels). Para obter uma melhor visualização dos dados, podemos aplicar uma tranformação de escala diferente.



Para alterar o tipo de escala, definiremos o atributo `scale`, usando o método `alt.Scale` e o parâmetro `type`.

Aqui está o resultado do uso de um tipo de escala de raiz quadrada (`sqrt`). As distâncias no intervalo de pixels agora correspondem à raiz quadrada das distâncias no domínio de dados.

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale=alt.Scale(type='sqrt'))
)

alt.Chart(...)

_Os pontos à esquerda agora estão melhor diferenciados, mas ainda vemos algumas distorções acentuadas._

Vamos tentar usar uma [escala logaritíma](https://en.wikipedia.org/wiki/Logarithmic_scale) (`log`):

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale=alt.Scale(type='log'))
)

alt.Chart(...)

_Agora os dados estão distribuídos de maneira muito mais uniforme e podemos ver as grandes diferenças nas concentrações necessárias para diferentes bactérias._

Em uma escala linear padrão, uma distância visual (pixel) de 10 unidades pode corresponder a uma *adição* de 10 unidades no domínio de dados. Uma transformação logarítmica mapeia entre multiplicação e adição, tal que `log(u) + log(v) = log(u*v)`. Como resultado, em uma escala logarítmica, uma distância visual de 10 unidades corresponde à *multiplicação* por 10 unidades no domínio de dados, assumindo um logaritmo de base 10. A escala `log` acima usa como padrão o logaritmo de base 10, mas podemos ajustar isso fornecendo um parâmetro `base` para a escala.


### Estilizando um Eixo
---

Dosagens mais baixas indicam maior eficácia. No entanto, algumas pessoas podem esperar que valores “melhores” estejam “para cima e para a direita” em um gráfico. Se quisermos atender a esta convenção, podemos inverter o eixo para codificar “eficácia” como uma escala MIC invertida.

Para fazer isso, podemos definir a propriedade de codificação `sort` como `'descending'`:

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'))
)

alt.Chart(...)

_Infelizmente o eixo está começando a ficar um pouco confuso: estamos plotando dados em escala logarítmica, no sentido inverso, e sem uma indicação clara de quais são nossas unidades!_

Vamos adicionar um título de eixo mais informativo: usaremos a propriedade `title` da codificação para fornecer o texto do título desejado:

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Neomycin MIC (μg/ml, reverse log scale)')
)

alt.Chart(...)

Muito melhor!

Por padrão, Altair coloca o eixo x na parte inferior do gráfico. Para alterar esses padrões, podemos adicionar um atributo `axis` com `orient='top'`:

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          axis=alt.Axis(orient='top'),
          title='Neomycin MIC (μg/ml, reverse log scale)')
)

alt.Chart(...)

Da mesma forma, o eixo y tem como padrão uma orientação `'esquerda'`, mas pode ser definido como `'direita'`.

### Comparando antibióticos: Ajustando Linhas de Grade, Contagens de Marcações e Dimensionamento
---

_Como a neomicina se compara a outros antibióticos, como estreptomicina e penicilina?_

Para começar a responder a essa pergunta, podemos criar gráficos de dispersão (_scatter plots_) , adicionando uma codificação do eixo y para outro antibiótico que espelhe o design do nosso eixo x para a neomicina.

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Streptomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Streptomycin MIC (μg/ml, reverse log scale)')
)

alt.Chart(...)

_Podemos ver que a neomicina e a estreptomicina parecem altamente correlacionadas, pois as cepas bacterianas respondem de forma semelhante a ambos os antibióticos._

Vamos continuar e comparar a neomicina com a penicilina:

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='Penicillin MIC (μg/ml, reverse log scale)')
)

alt.Chart(...)

_Agora vemos uma resposta mais diferenciada: algumas bactérias respondem bem à neomicina, mas não à penicilina, e vice-versa!_

Embora esse gráfico seja útil, podemos melhorá-lo. Os eixos x e y usam as mesmas unidades, mas têm extensões diferentes (a largura do gráfico é maior que a altura) e domínios diferentes (0,001 a 100 para o eixo x e 0,001 a 1.000 para o eixo y).

Vamos equalizar os eixos: podemos adicionar as configurações explícitas de comprimento e altura, `width` e `height`, do gráfico e especificar domínios correspondentes usando a propriedade `domain` de *scale*.

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='Penicillin MIC (μg/ml, reverse log scale)')
).properties(width=250, height=250)

alt.Chart(...)

_O gráfico resultante é mais equilibrado e menos propenso a interpretações errôneas sutis!_

No entanto, as linhas de grade agora são bem densas. Se quisermos remover as linhas de grade completamente, podemos adicionar `grid=False` ao atributo do eixo, o `axis`. Mas e se, em vez disso, quisermos reduzir o número de marcas de escala, por exemplo, incluindo apenas linhas de grade para cada ordem de magnitude?

Para alterar o número de marcações, podemos especificar a propriedade `tickCount` desejada para o objeto `Axis`. O `tickCount` é tratado como uma sugestão para o Altair, levando em conta outros fatores, como a utilização de intervalos agradáveis e intuitivos para humanos. Pode ser que não obtenhamos exatamente o número de marcações solicitado, mas o resultado será algo próximo.

In [ ]:
alt.Chart(antibiotics).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)')
).properties(width=250, height=250)

alt.Chart(...)

Ao definir o valor do `tickCount` como 5, obtemos o efeito desejado.

Nossos pontos do gráfico de dispersão (_scatter plot_) parecem um pouco pequenos. Vamos alterar o tamanho padrão definindo a propriedade `size` da marca do círculo. Esse valor de tamanho é a área da marca em pixels.

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
).properties(width=250, height=250)

alt.Chart(...)

Aqui, definimos a área de marcação do círculo para 80 pixels. `Ajuste ainda mais o valor como achar melhor!`

## Configurando as Cores da Legendas
---

### Definindo as Cores pela Coloração de Gram
---


_Acima vimos que a neomicina é mais eficaz para algumas bactérias, enquanto a penicilina é mais eficaz para outras. Mas como podemos dizer qual antibiótico usar se não conhecemos as espécies específicas de bactérias? A coloração de Gram serve como um diagnóstico para discriminar classes de bactérias!_

Vamos codificar `Gram_Staining` no canal `color` como um tipo de dado nominal:

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N')
).properties(width=250, height=250)

alt.Chart(...)

_Podemos ver que as bactérias Gram-positivas parecem mais suscetíveis à penicilina, enquanto a neomicina é mais eficaz para bactérias Gram-negativas!_

O esquema de cores acima foi escolhido automaticamente para fornecer cores perceptualmente distinguíveis para comparações nominais (iguais ou não). No entanto, podemos desejar personalizar as cores usadas. Neste caso, a coloração de Gram resulta em [colorações físicas distintas: rosa para Gram-negativo, roxo para Gram-positivo](https://cdn.kastatic.org/ka-perseus-images/4406e3cad301fa00abc08d17bf600a5bf0d481f5.png).

Vamos usar essas cores especificando um mapeamento de escala explícito do domínio dos dados, o `domain`, para a gama de cores, o `range`:

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple'])
    )
).properties(width=250, height=250)

alt.Chart(...)

Por padrão, as legendas são colocadas no lado direito do gráfico. Semelhante aos eixos, podemos alterar a orientação da legenda usando o parâmetro `orient`:

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=alt.Legend(orient='left')
    )
).properties(width=250, height=250)

alt.Chart(...)

Nós também podemos remover uma legenda completamente especificando `legend=None`:

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=None
    )
).properties(width=250, height=250)

alt.Chart(...)

### Cor por Espécie
---

Até aqui nós consideramos a efetividade dos antibióticos. Vamos mudar de direção e fazer uma pergunta diferente: o que a resposta do antibiótico nos ensina sobre as diferentes espécies de bactéria?

Para começar, vamos codificar `Bacteria` (um campo de dados nominal) usando o canal `color` (cor):

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:N')
).properties(width=250, height=250)

alt.Chart(...)

*O resultado está um pouco bagunçado!* Há espécies de bacteria o bastante para que o Altair comece a repetir as cores da sua paleta padrão de 10 cores para variáveis nominais.

Para usar cores personalizadas, nós podemos atualizar a codificação de cor da propriedade `scale` (escala). Uma opção é determinar explicitamente a escala dos valores `domain` (domínio) e `range` (intervalo) para indicar os mapeamentos exatos de cor por valor, como fizemos acima para a coloração de Gram. Uma outra opção é usar um esquema de cor alternativa. Está incluso no Altair uma variedade de esquemas integrados de cor. Para a lista completa, veja a [documentação do esquema de cores do Vega](https://vega.github.io/vega/docs/schemes/#reference).

Vamos tentar mudar paara um esquema integrado de 20 cores, `tableau20`, e defina isso usando a propriedade de escala do `scheme`.

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

*Nós agora temos uma única cor para cada bactéria, mas o gráfico ainda está uma bagunça. Entre outros problemas, a codificação não leva em conta as bactérias que pertencem ao mesmo gênero. No gráfico acima, as duas diferentes cepas de Salmonella apresentam matizes muito diferentes (verde-azulado e rosa), apesar de serem primos biológicos.*

Para testar um esquema diferente, nos também podemos mudar o tipo de dado de nominal para ordinal. O esquema ordinal padrão usa tons de azul, variando do claro ao escuro:

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:O')
).properties(width=250, height=250)

alt.Chart(...)

*Alguns desses tons de azul podem ser difíceis de distinguir.*

Para cores mais diferenciadas, podemos experimentar alternativas para o esquema de cores `blue` padrão. O esquema `viridis` varia tanto o matiz quanto a luminância.

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Bacteria:O',
          scale=alt.Scale(scheme='viridis'))
).properties(width=250, height=250)

alt.Chart(...)

*Bacterias do mesmo gênero agora têm cores mais similares que antes, mas o gráfico segue confuso. Há muitas cores, são difíceis de procurar com precisão na legenda, e duas bactérias podem ter cores similares mas gêneros diferentes.*

### Cor por Gênero
---

Vamos tentar colorir por gênero em vez de bactéria. Para isso, nós vamos adicionar uma transformação `calculate` que separa o nome da bactéria através dos caracteres de espaço e toma a primeira palavra do vetor resultante. Nós podemos então codificar o campo `Genus` resultante usando o esquema de cor `tableau20`.

(Note que a base de dados dos antibióticos inclui um campo `Genus` pré-calculado, mas nós vamos ignorar isto aqui em favor de explorar as transformações de dados do Altair.)

In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).transform_calculate(
    Genus='split(datum.Bacteria, " ")[0]'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Genus:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

*Humm... Enquanto os dados são melhor segregados por gênero, essa cacofonia de cores não parece particularmente útil.*

*Se olharmos cuidadosamente para alguns dos gráficos anteriores, podemos ver que apenas um punhado de bactérias tem o mesmo gênero compartilhado com outra bactéria: Salmonella, Staphylococcus, e Streptococcus. Para focar em nossa comparação, poderíamos adicionar cores apenas para esses valores de gêneros repetidos.*

Vamos adicionar uma outra transformação `calculate` que toma um nome de gênero, o mantém se ele é um dos valores repetidos, e caso contrário, utiliza a cadeia de caracteres `"Other"` ("Outro").

Além disso, podemos adicionar codificações personalizadas de cores usando vetores explícitos de `domain` (domínio) e `range` (intervalo) para a codificação de cores `scale` (escala).


In [ ]:
alt.Chart(antibiotics).mark_circle(size=80).transform_calculate(
  Split='split(datum.Bacteria, " ")[0]'
).transform_calculate(
  Genus='indexof(["Salmonella", "Staphylococcus", "Streptococcus"], datum.Split) >= 0 ? datum.Split : "Other"'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Neomycin MIC (μg/ml, reverse log scale)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='Penicillin MIC (μg/ml, reverse log scale)'),
    alt.Color('Genus:N',
          scale=alt.Scale(
            domain=['Salmonella', 'Staphylococcus', 'Streptococcus', 'Other'],
            range=['rgb(76,120,168)', 'rgb(84,162,75)', 'rgb(228,87,86)', 'rgb(121,112,110)']
          ))
).properties(width=250, height=250)

alt.Chart(...)

*Nós agora temos um gráfico muito mais revelador, tornado possível pela personalização dos eixos e legenda. Tire um momento para examinar o gráfico acima. Nota algum agrupamento surpreendente?*

*A região superior-esquerda contém um agrupamento de bactérias Streptococcus vermelhas, mas com uma outra cinza bactéria ao lado delas. Enquanto isso, em direção ao centro-direita nós vemos uma outra Streptococcus vermelha posicionada longe de suas "primas". Poderíamos esperar que bactérias do mesmo gênero (e portanto, presumivelmente mais geneticamente similares) fossem agrupadas mais próximas?*

Acontece que o conjunto de dados subjacente na verdade contém erros. O conjunto de dados reflete a designição de espécies usada no início dos anos 1950. No entanto, o consenso científico foi derrubado desde então. Aquele ponto cinza na esquerda-superior? Agora é considerado uma Streptococcus! Aquele ponto vermelho em direção ao centro-direita? Não é mais considerado uma Streptococcus!

É claro, por conta própria, este conjunto de dados não justifica totalmente essas reclassificações. No entanto, os dados contêm pistas biológicas valiosas que passaram despercebidas durante décadas! A visualização, quando usada por um observador devidamente qualificado e curioso, pode ser uma ferramenta poderosa para descoberta.

Este exemplo também reforça uma lição importante: ***seja sempre cético em relação aos seus dados!***

### Resistência antibiótica em cores
---


Podemos também usar o canal `color` (cor) para codificar valores quantitativos. No entanto, tenha em mente que a cor normalmente não mostra quantidades de forma tão eficaz quanto codificações de posição ou tamanho!

Abaixo está um mapa de calor básico dos valores de CIM de penicilina para cada bactéria. Vamos utilizar a marca `rect`(retângulo) e ordenar as bactérias em ordem decrescente de valores em CIM (da mais resistente para a menos resistente).

In [ ]:
alt.Chart(antibiotics).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending')
    ),
    alt.Color('Penicillin:Q')
)

alt.Chart(...)

Podemos melhorar ainda mais este gráfico combinando os recursos que vimos até então: uma escala transformada em logaritmo, uma mudança na orientação do eixo, um esquema de cores personalizado (`plasma`), ajuste da contagem de marcas e texto personalizado no título. Também vamos utilizar opções de configuração para ajustar a posição do título do eixo e o alinhamento do título da legenda.

In [ ]:
alt.Chart(antibiotics).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(
        orient='right',     # orienta o eixo para o lado direito no gráfico
        titleX=7,           # define a posição de x para 7 pixels à direita do gráfico
        titleY=-2,          # define a posição de y para 2 pixels acima do gráfico
        titleAlign='left',  # alinha o texto à esquerda
        titleAngle=0        # desfaz a orientação padrão do título
      )
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='Penicillin MIC (μg/ml)'
    )
)

alt.Chart(...)

Ou podemos simplesmente remover o título do eixo e usar a propriedade `title` para adicionar um título para o gŕafico inteiro:

In [ ]:
alt.Chart(antibiotics, title='Penicillin Resistance of Bacterial Strains').mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(orient='right', title=None)
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='Penicillin MIC (μg/ml)'
    )
).configure_title(
  anchor='start', # ancora e alinha o título à esquerda
  offset=5        # define o espaçamento do título no gráfico

alt.Chart(...)

## Resumo
---

Juntando tudo o que aprendemos nesses capítulos sobre codificações, transformação de dados e customização, você está apto a realizar uma grande variedade de gráficos estatísticos. Agora você pode usar o Altair no dia a dia para explorar e comunicar dados!

Interessado em aprender mais sobre este assunto?


*   Comece pela [documentação de Customização de Visualizações do Altair](https://altair-viz.github.io/user_guide/customization.html).
*   Para uma discussão complementar sobre mapeamento de escala, veja ["Introducing d3-scale"](https://medium.com/@mbostock/introducing-d3-scale-61980c51545f) (em português, "Introduzindo a escala d3").
*   Para uma exploração mais detalhada de todas as formas como os eixos e as legendas podem ser customizadas com a biblioteca Vega (que é a base do Altair e do Vega-Lite), veja ["A Guide to Guides: Axes & Legends in Vega"](https://observablehq.com/@vega/a-guide-to-guides-axes-legends-in-vega) (em português, "Um guia para guias: eixos e legendas em Vega").
*   Para uma fascinante história sobre o dataset de antibióticos, veja ["That's Funny..." de Wainer e Lysen](https://www.americanscientist.org/article/thats-funny) (em português, "É Engraçado...") no *American Scientist*.